In [ ]:
!nvidia-smi

In [ ]:
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import json
import matplotlib.patches as patches
import os
import glob
from tqdm import tqdm

In [ ]:
data_folder = '/root/data/data_quality/dataset/'

## #0 Create dataset

In [ ]:
# json_path = '/root/data/aquabyte-images/cocofiles/coco_visibility_2018-10-01.json'
# example_coco = COCO(json_path)

In [ ]:
# for (image_id, image_data) in tqdm(example_coco.imgs.items()):
#     annotation_ids = example_coco.getAnnIds(imgIds=image_data['id']) #, catIds=category_ids, iscrowd=None)
#     annotations = example_coco.loadAnns(annotation_ids)
#     image = io.imread(image_data['local_path'])
#     for annotation in annotations:
#         crop_id = '{}_{}.jpg'.format(image_id, annotation['id'])
#         bbox = annotation['bbox']
#         y1, x1, width, height = [int(b) for b in bbox]
#         # print(bbox)
#         crop = image[x1:x1+height, y1:y1+width, :]
#         io.imsave(os.path.join(data_folder, crop_id), crop)

## #1 Look at the dataset

In [ ]:
crops = glob.glob(data_folder + '/*.jpg')

In [ ]:
for _ in range(10):
    plt.imshow(io.imread(np.random.choice(crops)))
    plt.show()

## #2 Create a data generator

In [ ]:
import keras
import cv2
from albumentations import OneOf, MotionBlur, Blur, MedianBlur, Compose, RandomBrightness, RandomGamma, GaussNoise, RandomContrast
# import albumentations as A

In [ ]:
image_size = [224, 224, 3]
batch_size = 32
augmentation = Compose([OneOf([MotionBlur(p=1/3.0), 
                               Blur(p=1/3.0), 
                               MedianBlur(p=1/3.0)], p=1.0), 
                        OneOf([RandomBrightness(limit=1.0, p=1/3.0),
                               RandomGamma(gamma_limit=(60, 150), p=1/3.0),
                               RandomContrast(limit=1.0, p=1/3.0)
                              ], p=1.0),
                        GaussNoise(var_limit=(30, 50), p=0.5)],
                       p=1.0)

In [ ]:
# random.seed(42)
# image = cv2.imread(crops[0])

# light = A.Compose([
# #    A.RandomBrightness(limit=3.0, p=1),
# #     A.RandomContrast(limit=1.0, p=1),
# #     A.RandomGamma(p=1),
# #     A.RGBShift(),
# #    A.CLAHE(p=1),
# #     A.ToGray(),
# #     A.HueSaturationValue(),
# ], p=1)

# medium = A.Compose([
#     A.CLAHE(p=1),
#     A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=50, val_shift_limit=50, p=1),
# ], p=1)


# strong = A.Compose([
#     A.ChannelShuffle(p=1),
# ], p=1)
# for _ in range(10):
#     aug_img = light(image=image)['image']
#     plt.figure(figsize=(10, 20))
#     plt.imshow(aug_img)
#     plt.show()

In [ ]:
def generator(image_paths, image_size, batch_size, steps_per_epoch, augmentation):
    i = 0
    while True:
        x_batch = np.zeros((batch_size, image_size[0], image_size[1], image_size[2]), dtype=np.uint8)
        y_batch = np.zeros((batch_size))
        for (ind, j) in enumerate(range(i*batch_size, (i+1)*batch_size)):
            path = image_paths[j]
            image = io.imread(path)
            image = cv2.resize(image, (image_size[0], image_size[1]))
            coin = np.random.rand()
            if augmentation is not None:
                if coin > 0.5:
                    image = augmentation(image=image)['image']
                    y_batch[ind, ...] += 1
            x_batch[ind, ...] = image 
        i += 1
        if i >= steps_per_epoch:
            i = 0
        yield x_batch, keras.utils.to_categorical(y_batch, num_classes=2)

In [ ]:
train_generator = generator(crops, image_size, batch_size, 10, augmentation)

In [ ]:
xb, yb = next(train_generator)

In [ ]:
for i in range(batch_size):
    plt.imshow(xb[i, ...].squeeze())
    k = np.argmax(yb[i, :])
    if k == 0:
        plt.title('Clear')
    else:
        plt.title('Blurry')
    plt.show()

## #3 Train

In [ ]:
from keras.applications.mobilenet import MobileNet
import random
from keras.optimizers import Adam
from keras.models import Model
from keras import layers

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [ ]:
mbnet = MobileNet(input_shape=image_size, dropout=1e-3, include_top=False, weights='imagenet', classes=2)

In [ ]:
shape = (1, 1, int(1024 * 1.0))

In [ ]:
x = layers.GlobalAveragePooling2D()(mbnet.output)
x = layers.Reshape(shape, name='reshape_1')(x)
x = layers.Dropout(1e-3, name='dropout')(x)
x = layers.Conv2D(2, (1, 1),
                  padding='same',
                  name='conv_preds')(x)
x = layers.Activation('softmax', name='act_softmax')(x)
x = layers.Reshape((2,), name='reshape_2')(x)

In [ ]:
model = Model([mbnet.input], [x])

In [ ]:
random.seed(18679)
random.shuffle(crops)

In [ ]:
cutoff = int(len(crops)*0.8)
train = crops[:cutoff]
val = crops[cutoff:]
print("Train set: {}".format(len(train)))
print("Val set: {}".format(len(val)))

In [ ]:
steps_per_epoch = len(train) // batch_size
steps_per_epoch_val = len(val) // batch_size

In [ ]:
train_generator = generator(train, image_size, batch_size, steps_per_epoch, augmentation)
val_generator = generator(val, image_size, batch_size, steps_per_epoch_val, augmentation)

In [ ]:
x,y = next(train_generator)

In [ ]:
x.shape

In [ ]:
y.shape

In [ ]:
lr=0.001
adam = Adam(lr=lr)
model.compile(adam, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
# start training# start 
history = model.fit_generator(
        generator=train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=10,
        verbose=1,
        # callbacks=[saveh, lr_scheduler, checkpoint],
        validation_data= val_generator,
        validation_steps= steps_per_epoch_val)

## #4 Val

In [ ]:
xb, yb = next(val_generator)

In [ ]:
y_pred = model.predict_on_batch(xb)

In [ ]:
for i in range(batch_size):
    k = np.argmax(y_pred[i, :])
    plt.imshow(xb[i, ...])
    if k == 0:
        plt.title('Clear')
    else:
        plt.title('Blurry')
    plt.show()
        

## #5 Test

In [ ]:
frames = glob.glob('/root/data/data_quality/crops/next/*/input/left_frame.jpg.crop.jpg')

In [ ]:
test_generator = generator(frames, image_size, batch_size, 10, augmentation=None)

In [ ]:
for _ in range(2):
    xb, _ = next(test_generator)
    y_pred = model.predict_on_batch(xb)
    f, ax = plt.subplots(8, 4, figsize=(10, 40))
    for i in range(batch_size):
        k = np.argmax(y_pred[i, :])

        row = i // 4
        col = i % 4
        ax[row, col].imshow(xb[i, ...])
        if k == 0:
            ax[row, col].set_title('Clear')
        else:
            ax[row, col].set_title('Blurry')
    plt.show()